# Server Tutorial (CLI and Python)

This tutorial covers options 6-9: serving data via terminal commands or Python API.

## Server Options Overview

| # | Method | Data Format | Python Required | Lazy Loading | Best For |
|---|--------|-------------|-----------------|--------------|----------|
| 6 | `cellucid serve` | Exported | Yes | Yes | Pre-exported data |
| 7 | `cellucid serve-anndata` | h5ad/zarr | Yes | Yes | Quick h5ad/zarr serving |
| 8 | `serve()` | Exported | Yes | Yes | Python integration |
| 9 | `serve_anndata()` | h5ad/zarr/memory | Yes | Yes | Flexible AnnData |

## When to Use

- Team collaboration (share via SSH tunnel)
- Remote server with large datasets
- Development and testing
- Integration with analysis pipelines

In [ ]:
# Setup
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

---

# Option 6: CLI - Serve Pre-exported Data

| Property | Value |
|----------|-------|
| Data Format | Pre-exported binary |
| Python Required | Yes (server) |
| Lazy Loading | Yes |
| Performance | Best |

## Usage

```bash
# Basic usage
cellucid serve /path/to/export_dir

# With options
cellucid serve /path/to/export_dir --port 9000 --no-browser
```

## CLI Options

| Option | Description | Default |
|--------|-------------|---------|
| `--port, -p` | Port to serve on | 8765 |
| `--host, -H` | Host to bind to | 127.0.0.1 |
| `--no-browser` | Don't auto-open browser | False |
| `--quiet, -q` | Suppress info messages | False |

---

# Option 7: CLI - Serve h5ad or zarr Directly

| Property | Value |
|----------|-------|
| Data Format | h5ad or zarr |
| Python Required | Yes (server) |
| Lazy Loading | **Yes** (backed mode for h5ad, chunked for zarr) |
| Performance | Good |

## Usage

```bash
# Serve h5ad file
cellucid serve-anndata /path/to/data.h5ad

# Serve zarr store (auto-detected)
cellucid serve-anndata /path/to/data.zarr

# With options
cellucid serve-anndata /path/to/data.h5ad --port 9000 --no-browser
```

## CLI Options

| Option | Description | Default |
|--------|-------------|---------|
| `--port, -p` | Port to serve on | 8765 |
| `--host, -H` | Host to bind to | 127.0.0.1 |
| `--no-browser` | Don't auto-open browser | False |
| `--quiet, -q` | Suppress info messages | False |
| `--no-backed` | Load entire file into memory | False (lazy load) |
| `--latent-key` | Key in obsm for latent space | Auto-detected |

## Lazy Loading Explained

By default, `cellucid serve-anndata` opens files in **lazy mode**:
- **h5ad**: Opens in AnnData backed mode (data read from disk on-demand)
- **zarr**: Uses zarr's native chunked access (efficient partial reads)
- Low memory footprint for both formats
- Perfect for large files

Use `--no-backed` only if:
- File is small and you want maximum query speed
- File is on slow storage (network drive)

## zarr Format Notes

- zarr stores can be directories (`.zarr/`) or consolidated files
- The server auto-detects the format based on file structure
- zarr is especially efficient for cloud storage (S3, GCS) scenarios

---

# Option 8: Python - Serve Pre-exported Data

| Property | Value |
|----------|-------|
| Data Format | Pre-exported binary |
| Python Required | Yes |
| Lazy Loading | Yes |
| Performance | Best |

In [ ]:
from cellucid import serve

# Serve pre-exported data (blocking - Ctrl+C to stop)
# serve("/path/to/export_dir", port=8765)

# Or use CellucidServer for more control
from cellucid import CellucidServer

# server = CellucidServer("/path/to/export_dir", port=8765)
# server.start_background()  # Non-blocking
# print(f"Server running at {server.url}")
# ... do other things ...
# server.stop()

print("Use serve() or CellucidServer for pre-exported data")

---

# Option 9: Python - Serve h5ad, zarr, or In-Memory AnnData

| Property | Value |
|----------|-------|
| Data Format | h5ad, zarr, or in-memory AnnData |
| In-Memory AnnData | **Yes** |
| h5ad/zarr File Support | **Yes** |
| Python Required | Yes |
| Lazy Loading | Yes (backed mode for h5ad, chunked for zarr) |
| Performance | Good |

In [ ]:
from cellucid import serve_anndata

# Serve h5ad file (lazy loading by default)
# serve_anndata("/path/to/data.h5ad", port=8765)

# Serve zarr store (lazy loading by default)
# serve_anndata("/path/to/data.zarr", port=8765)

# Serve in-memory AnnData
# serve_anndata(adata, port=8765)

# With options
# serve_anndata(
#     "/path/to/data.h5ad",  # or .zarr
#     port=8765,
#     open_browser=True,
#     latent_key="X_pca",  # For outlier quantile computation
# )

print("Use serve_anndata() for h5ad files, zarr stores, or in-memory AnnData")

In [ ]:
# Using AnnDataServer for more control
from cellucid import AnnDataServer

# For h5ad
# server = AnnDataServer("/path/to/data.h5ad", port=8765)
# server.start_background()  # Non-blocking
# print(f"Server running at {server.url}")
# print(f"Viewer URL: {server.viewer_url}")

# For zarr
# server = AnnDataServer("/path/to/data.zarr", port=8766)
# server.start_background()

# ... do other things ...
# server.stop()

print("Use AnnDataServer for non-blocking server control (works with h5ad and zarr)")

---

# Remote Server Access

To access data on a remote server, use SSH port forwarding.

## Setup

### On Remote Server

```bash
# Option 1: Serve h5ad directly
cellucid serve-anndata /data/large_dataset.h5ad --port 8765 --no-browser

# Option 2: Serve zarr directly
cellucid serve-anndata /data/large_dataset.zarr --port 8765 --no-browser

# Option 3: Serve pre-exported data
cellucid serve /data/my_export --port 8765 --no-browser

# Option 4: Python script
python serve_my_data.py
```

### On Local Machine

```bash
# Create SSH tunnel
ssh -L 8765:localhost:8765 user@remote-server

# Keep terminal open while using the tunnel
```

### Access in Browser

```
# For pre-exported data:
https://cellucid.com?remote=http://localhost:8765

# For h5ad/zarr (adds performance warning):
https://cellucid.com?remote=http://localhost:8765&anndata=true
```

---

# Example: Complete Remote Workflow

## With h5ad

```bash
# Terminal 1: SSH to server
ssh user@server.edu

# On server: Start Cellucid
cellucid serve-anndata /data/analysis/pbmc_500k.h5ad --port 8765 --no-browser

# Output:
# ============================================================
#   Cellucid AnnData Server
# ============================================================
#   Data source:    /data/analysis/pbmc_500k.h5ad
#   Cells:          500,000
#   Genes:          25,000
#   Dimensions:     [3, 2]
#   Backed mode:    True
#   Server URL:     http://127.0.0.1:8765
# ...
```

## With zarr

```bash
# On server: Start Cellucid with zarr
cellucid serve-anndata /data/analysis/pbmc_500k.zarr --port 8765 --no-browser

# Output:
# ============================================================
#   Cellucid AnnData Server
# ============================================================
#   Data source:    /data/analysis/pbmc_500k.zarr
#   Cells:          500,000
#   Genes:          25,000
#   Format:         zarr
#   Server URL:     http://127.0.0.1:8765
# ...
```

```bash
# Terminal 2: Create SSH tunnel (local machine)
ssh -L 8765:localhost:8765 user@server.edu

# Keep this terminal open
```

```
# Browser: Open the viewer
https://cellucid.com?remote=http://localhost:8765&anndata=true
```

---

# Performance Comparison

| Method | Initial Load | Gene Query | Memory (Server) |
|--------|--------------|------------|------------------|
| `cellucid serve` (exported) | Fast | Fast | Low |
| `cellucid serve-anndata` h5ad (backed) | Medium | Medium | Low |
| `cellucid serve-anndata` zarr | Medium | Medium | Low |
| `cellucid serve-anndata --no-backed` | Slow | Fast | High |
| `serve_anndata(adata)` | Instant | Fast | As needed |

### Recommendations

| Scenario | Recommendation |
|----------|----------------|
| Production, sharing | Pre-export + `cellucid serve` |
| Large h5ad files | `cellucid serve-anndata data.h5ad` (backed) |
| Large zarr stores | `cellucid serve-anndata data.zarr` |
| Small files, repeated queries | `cellucid serve-anndata --no-backed` |
| In-memory analysis | `serve_anndata(adata)` |
| Cloud storage (S3, GCS) | zarr format recommended |

---

# Troubleshooting

### "Port already in use"

```bash
# Find what's using the port
lsof -i :8765

# Use a different port
cellucid serve-anndata data.h5ad --port 9000
```

### "Connection refused" via SSH tunnel

1. Verify server is running on remote: `curl http://localhost:8765/_cellucid/health`
2. Check tunnel is active
3. Ensure same port on both sides

### Slow gene loading

Expected for h5ad serving without quantization. For faster queries:
- Pre-export with `prepare(quantize_var=True)`
- Or use `--no-backed` for in-memory caching (requires more RAM)

### Server stops after closing terminal

Use `nohup` or `screen` for persistent servers:

```bash
# With nohup
nohup cellucid serve-anndata data.h5ad --no-browser > server.log 2>&1 &

# With screen
screen -S cellucid
cellucid serve-anndata data.h5ad --no-browser
# Ctrl+A, D to detach
```

---

## Summary

### CLI Commands

| Command | Use Case |
|---------|----------|
| `cellucid serve <dir>` | Pre-exported data |
| `cellucid serve-anndata <file>` | h5ad or zarr file directly |

### Python API

| Function | Use Case |
|----------|----------|
| `serve(dir)` | Pre-exported data |
| `serve_anndata(data)` | h5ad, zarr, or in-memory AnnData |
| `CellucidServer(dir)` | Non-blocking pre-exported server |
| `AnnDataServer(data)` | Non-blocking AnnData server (h5ad/zarr) |

### Remote Access

1. Start server on remote: `cellucid serve-anndata data.h5ad --no-browser` (or `.zarr`)
2. Create SSH tunnel: `ssh -L 8765:localhost:8765 user@server`
3. Open: `https://cellucid.com?remote=http://localhost:8765`

---

## Next: [05_jupyter_tutorial.ipynb](05_jupyter_tutorial.ipynb)